# import dependancies

In [269]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler


In [270]:
df = pd.read_csv('data/raw/dataset.csv')
df.columns

Index(['Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Exited',
       'CreditScoreBins'],
      dtype='object')

In [271]:
remainder_features = ['NumOfProducts','HasCrCard','IsActiveMember']
numerical_features=['Age','Tenure','Balance','EstimatedSalary']
nominal_features = ['Gender','Geography']
ordinal_features = ['CreditScoreBins']

In [272]:
numerical_transformer = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

In [273]:
nominal_transformer = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='constant',
        fill_value='missing')),
        ('encoder',OneHotEncoder())
    ]
)

In [274]:
ordinal_transformer = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='constant',
        fill_value='missing')),
        ('encoder',OrdinalEncoder())
    ]
)

In [275]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num',numerical_transformer,numerical_features),
        ('nom',nominal_transformer,nominal_features),
        ('ord',ordinal_transformer,ordinal_features),
    ],
    remainder='drop'
)





In [276]:
nominal_feature_name = []

for feature in nominal_features:
    unique_values = df[feature].unique()
    nominal_feature_name.extend([f"{feature}_{val}" for val in unique_values])






In [277]:
df_cp = df.copy()


In [278]:
df_transformed = pd.DataFrame(
    preprocessor.fit_transform(df_cp),
    columns=numerical_features+nominal_feature_name+ordinal_features
)

df_remainder = df[remainder_features]


df_final = pd.concat(
    [df_transformed,df_remainder,df_cp.Exited],
    axis=1
)

df_final.head()



,Age,Tenure,Balance,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Spain,Geography_Germany,CreditScoreBins,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,0.428433,-1.041526,-1.225704,0.021852,1.0,0.0,1.0,0.0,0.0,1.0,1,1,1,1
1,0.332317,-1.387294,0.117331,0.216474,1.0,0.0,0.0,0.0,1.0,1.0,1,0,1,0
2,0.428433,1.033083,1.332886,0.240624,1.0,0.0,1.0,0.0,0.0,3.0,3,1,0,1
3,0.124038,-1.387294,-1.225704,-0.108935,1.0,0.0,1.0,0.0,0.0,2.0,2,0,0,0
4,0.522339,-1.041526,0.785627,-0.365260,1.0,0.0,0.0,0.0,1.0,0.0,1,1,1,0


In [279]:
df_final.to_csv('data/processed/x_transformed.csv')

## Handle Class Imbalance

1st split data, then make inbalance.

In [280]:
X = df_final.drop(columns=['Exited'])
Y = df_final['Exited']
